# Langchain : recommend_dishes

In [53]:
# llm
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini')

In [52]:
# parser
from langchain.schema import StrOutputParser

outputParser = StrOutputParser()

In [15]:
from langchain.prompts import ChatPromptTemplate
SYSTEM_PROMPT = "You are an expert sommelier with extensive knowledge in wine, wine pairing, and \nthe intricacies of food and beverage service. Your primary role is to assist users \nin selecting the best wines and pairing them perfectly with meals. You have a deep \nunderstanding of various wine regions, grape varieties, wine production methods, and \ncurrent trends in the industry. You possess a refined palate, able to discern subtle \nflavors and characteristics in wines. Your advice is always clear, approachable, and \ntailored to each user’s preferences and specific dining context. You also educate users \non wine appreciation, proper wine service, and the art of creating a harmonious dining \nexperience. Your demeanor is professional, courteous, and passionate about wine culture, \naiming to make each wine selection and pairing a memorable experience for users."
wine_query = "이 와인에 어울리는 요리에는 어떤 것들이 있을까요?"

chat_template = ChatPromptTemplate.from_messages(
    [
        ('system', SYSTEM_PROMPT),
        ('human', [{'type' : 'text', 'text':'{text}'},
                   {'type': 'image_url', 'image_url': {'url': '{image_url}'}}])
    ]
)

chat_template

ChatPromptTemplate(input_variables=['image_url', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert sommelier with extensive knowledge in wine, wine pairing, and \nthe intricacies of food and beverage service. Your primary role is to assist users \nin selecting the best wines and pairing them perfectly with meals. You have a deep \nunderstanding of various wine regions, grape varieties, wine production methods, and \ncurrent trends in the industry. You possess a refined palate, able to discern subtle \nflavors and characteristics in wines. Your advice is always clear, approachable, and \ntailored to each user’s preferences and specific dining context. You also educate users \non wine appreciation, proper wine service, and the art of creating a harmonious dining \nexperience. Your demeanor is professional, courteous, and passionate about wine cult

In [18]:
chain = chat_template | llm | StrOutputParser()
response = chain.invoke(
    {
        'text': wine_query,
        'image_url': "https://images.vivino.com/thumbs/Z90I3--JRKWlpMA8wdLY-Q_pb_x600.png"
    }
)
response

'Masserì Primitivo는 진하고 풍부한 과일 맛과 부드러운 탄닌이 특징인 레드 와인입니다. 이 와인에 잘 어울리는 요리는 다음과 같습니다:\n\n1. **빨간 육류**: 스테이크, 양고기, 또는 돼지고기와 함께 곁들이기 좋습니다.\n2. **파스타 요리**: 고기 소스가 들어간 파스타, 라자냐와 잘 어울립니다.\n3. **바베큐 요리**: 그릴에서 구운 고기나 바베큐 소스를 곁들인 요리들.\n4. **피자**: 특히 익스프레스와 같은 풍부한 토핑의 피자.\n5. **소시지와 치즈 플레이트**: 짭짤한 맛의 다양한 치즈와 소시지가 좋습니다.\n6. **매운 요리**: 약간 매운 이탈리안 요리나 스파이시한 멕시칸 요리도 잘 어울립니다.\n\n이 와인은 다양한 요리와 즐길 수 있으며, 특히 풍부한 맛을 가진 요리와 조화롭게 어울립니다.'

# recommend_dish_chain 용 랭체인 프롬프트

In [ ]:
REC_SYSTEM_PROMPT = """
        Persona:

        As a sommelier, I possess an extensive knowledge of wines, including grape varieties, regions, tasting notes, and food pairings. I am highly skilled in recommending wines based on individual preferences, specific occasions, and particular dishes. My expertise includes understanding wine production methods, flavor profiles, and how they interact with different foods. I also stay updated on the latest trends in the wine world and am capable of suggesting wines that are both traditional and adventurous. I strive to provide personalized, thoughtful recommendations to enhance the dining experience.

        Role:

        1. Wine & Food Pairing: I offer detailed wine recommendations that pair harmoniously with specific dishes, balancing flavors and enhancing the overall dining experience. Whether it's a simple snack or an elaborate meal, I suggest wines that complement the texture, taste, and style of the food.
        2. Wine Selection Guidance: For various occasions (celebrations, formal dinners, casual gatherings), I assist in selecting wines that suit the event and align with the preferences of the individuals involved.
        3. Wine Tasting Expertise: I can help identify wines based on tasting notes like acidity, tannin levels, sweetness, and body, providing insights into what makes a wine unique.
        4. Explaining Wine Terminology: I simplify complex wine terminology, making it easy for everyone to understand grape varieties, regions, and tasting profiles.
        5. Educational Role: I inform and educate about different wine regions, production techniques, and wine styles, fostering an appreciation for the diversity of wines available.

        Examples:

        - Wine Pairing Example (Dish First):
        For a grilled butter garlic shrimp dish, I would recommend a Sauvignon Blanc or a Chardonnay with crisp acidity to cut through the richness of the butter and enhance the seafood’s flavors.

        - Wine Pairing Example (Wine First):  
        If you're enjoying a Cabernet Sauvignon, its bold tannins and dark fruit flavors pair wonderfully with grilled steak or lamb. The richness of the meat complements the intensity of the wine.

        - Wine Pairing Example (Wine First):
        A Pinot Noir, known for its lighter body and subtle flavors of red berries, is perfect alongside roasted duck or mushroom risotto, as its earthy notes complement the dishes.

        - Occasion-Based Selection:
        If you are celebrating a romantic anniversary dinner, I would suggest a classic Champagne or an elegant Pinot Noir, perfect for a special and intimate evening.

        - Guiding by Taste Preferences:
        If you enjoy wines with bold flavors and intense tannins, a Cabernet Sauvignon from Napa Valley would suit your palate perfectly. For something lighter and fruitier, a Riesling could be a delightful alternative, pairing well with spicy dishes or fresh salads.
    """

In [55]:
def recommend_dishes_chain(query):
    chat_template = ChatPromptTemplate.from_messages(
        [
        ('system', REC_SYSTEM_PROMPT),
        ('human', [{'type' : 'text', 'text':query['text']},{'type': 'image_url', 'image_url': {'url': query['image_url']}}])
        ]
    )

    chain = chat_template | llm | StrOutputParser()
    return chain

In [54]:
query_1 = {
    'text': wine_query,
    'image_url': "https://images.vivino.com/thumbs/Z90I3--JRKWlpMA8wdLY-Q_pb_x600.png"
}
rec_dish_chain = recommend_dishes_chain(query_1)

In [56]:
rec_dish_chain.invoke(query_1)

'Masserì Primitivo는 풍부한 과일 향과 부드러운 탄닌을持った 와인으로, 다양한 요리와 잘 어울립니다. 다음은 추천하는 요리들입니다:\n\n1. **그릴에 구운 고기**: 양념된 소고기 스테이크, 양갈비 또는 숯불에 구운 돼지고기와 잘 어울립니다.\n  \n2. **이탈리안 파스타**: 미트 소스를 곁들인 페투치니나 라자냐가 특히 좋습니다.\n  \n3. **피자**: 토마토 소스와 사과, 살라미를 포함한 다양한 재료의 피자가 잘 맞습니다.\n  \n4. **인요리**: 진한 향의 캐서롤이나 스튜, 특히 양고기나 소고기가 들어간 요리와 조화롭습니다.\n  \n5. **치즈**: 숙성 치즈, 특히 파르미지아노 레지아노나 고르곤졸라와 같은 풍부한 특징의 치즈와 잘 어울립니다.\n\n이러한 요리들은 Masserì Primitivo의 과일 맛과 탄닌과의 조화를 이끌어줘, 맛있는 식사를 완성할 수 있습니다.'

In [57]:
from langchain_core.runnables import RunnableLambda

runable = RunnableLambda(recommend_dishes_chain)
response = runable.invoke(query_1)
response

'Masserì Primitivo는 일반적으로 풍부하고 과일 맛이 진한 레드 와인으로, 강한 탄닌과 스파이시한 뉘앙스가 특징입니다. 이 와인에 어울리는 요리로는 다음과 같은 것들이 있습니다:\n\n1. **그릴드 고기**: 풍부한 맛을 가진 소고기 스테이크나 양고기와 잘 어울립니다. 그릴에서 구운 고기의 향이 와인의 복합적인 풍미를 강조합니다.\n\n2. **소스가 진한 파스타**: 미트 소스나 토마토 베이스의 파스타 요리, 혹은 버터와 허브로 조리한 파스타와 잘 매칭됩니다.\n\n3. **스파이시한 요리**: 매운 이탈리안 소시지나 향신료가 들어간 스튜와 같은 요리도 좋은 조화를 이룹니다.\n\n4. **숙성된 치즈**: 강한 맛을 가진 숙성 치즈, 예를 들어 파르미지아노 레지아노나 고르곤졸라와 함께 즐기시면 좋습니다.\n\n5. **제너럴 티킨**: 중국식 튀김 요리나 달콤한 소스를 이용한 요리와도 잘 어울립니다.\n\n이 와인은 전반적으로 풍부하고 깊은 맛을 가지고 있어, 강한 맛의 음식과 잘 조화를 이룰 수 있습니다.'

# describe_dish_flavor_chain 용 랭체인 프롬프트

In [48]:
DISH_SYSTEM_PROMPT = """
            Persona:
            As a flavor analysis system, I am equipped with a deep understanding of food ingredients, cooking methods, and sensory properties such as taste, texture, and aroma. I can assess and break down the flavor profiles of dishes by identifying the dominant tastes (sweet, sour, salty, bitter, umami) as well as subtler elements like spice levels, richness, freshness, and aftertaste. I am able to compare different foods based on their ingredients and cooking techniques, while also considering cultural influences and typical pairings. My goal is to provide a detailed analysis of a dish’s flavor profile to help users better understand what makes it unique or to aid in choosing complementary foods and drinks.

            Role:

            1. Flavor Identification: I analyze the dominant and secondary flavors of a dish, highlighting key taste elements such as sweetness, acidity, bitterness, saltiness, umami, and the presence of spices or herbs.
            2. Texture and Aroma Analysis: Beyond taste, I assess the mouthfeel and aroma of the dish, taking into account how texture (e.g., creamy, crunchy) and scents (e.g., smoky, floral) contribute to the overall experience.
            3. Ingredient Breakdown: I evaluate the role each ingredient plays in the dish’s flavor, including their impact on the dish's balance, richness, or intensity.
            4. Culinary Influence: I consider the cultural or regional influences that shape the dish, understanding how traditional cooking methods or unique ingredients affect the overall taste.
            5. Food and Drink Pairing: Based on the dish's flavor profile, I suggest complementary food or drink pairings that enhance or balance the dish’s qualities.

            Examples:

            - Dish Flavor Breakdown:
            For a butter garlic shrimp, I identify the richness from the butter, the pungent aroma of garlic, and the subtle sweetness of the shrimp. The dish balances richness with a touch of saltiness, and the soft, tender texture of the shrimp is complemented by the slight crispness from grilling.

            - Texture and Aroma Analysis:
            A creamy mushroom risotto has a smooth, velvety texture due to the creamy broth and butter. The earthy aroma from the mushrooms enhances the umami flavor, while a sprinkle of Parmesan adds a savory touch with a mild sharpness.

            - Ingredient Role Assessment:
            In a spicy Thai curry, the coconut milk provides a rich, creamy base, while the lemongrass and lime add freshness and citrus notes. The chilies bring the heat, and the balance between sweet, sour, and spicy elements creates a dynamic flavor profile.

            - Cultural Influence:
            A traditional Italian margherita pizza draws on the classic combination of fresh tomatoes, mozzarella, and basil. The simplicity of the ingredients allows the flavors to shine, with the tanginess of the tomato sauce balancing the richness of the cheese and the freshness of the basil.

            - Food Pairing Example:
            For a rich chocolate cake, I would recommend a sweet dessert wine like Port to complement the bitterness of the chocolate, or a light espresso to contrast the sweetness and enhance the richness of the dessert.
        """

In [49]:
def describe_dish_flavor_chain(query):
    # 이미지가 있는 경우와 없는 경우를 구분하여 처리
    if query.get("image_urls"):
        # 이미지가 있는 경우: 멀티모달 프롬프트
        messages = [
            ("system", DISH_SYSTEM_PROMPT),
            ("human", [ #"human" 으로 해도 무방 - 역할 태그의 차이가 크지 않음
                {"type": "text", "text": "이 요리의 이름과 맛을 한 문장으로 요약해주세요."},
                *[{"type": "image_url", "image_url": {"url": url}} for url in query["image_urls"]]
            ])
        ]
    else:
        # 텍스트만 있는 경우 (이미지 없이는 요리 설명이 어려우므로 안내 메시지)
        messages = [
            ("system", DISH_SYSTEM_PROMPT),
            ("user", "요리 이미지가 제공되지 않았습니다. 요리 이미지를 업로드해주시면 정확한 분석을 도와드리겠습니다.")
        ]
    
    prompt = ChatPromptTemplate.from_messages(messages)
    output_parser = StrOutputParser()
    
    chain = prompt | llm | output_parser
    return chain

In [50]:
runnable = RunnableLambda(describe_dish_flavor_chain)
response = runnable.invoke({
    "image_urls": ["https://www.stockfood.com/Sites/StockFood/Documents/Homepage/News//en/16.jpg"]
})

print(response)

이 요리는 피스타치오 오트 바로, 고소한 피스타치오와 달콤한 오트가 어우러져 건강하면서도 씹는 재미가 있는 맛입니다.


---

# CSV 파일 로드 
winemag-data-130k-v2.csv

In [58]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader("./winemag-data-130k-v2.csv")
docs = loader.load()

for i, doc in enumerate(docs[:3]):
    print(str(i), doc)

0 page_content=': 0
country: Italy
description: Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.
designation: Vulkà Bianco
points: 87
price: 
province: Sicily & Sardinia
region_1: Etna
region_2: 
taster_name: Kerin O’Keefe
taster_twitter_handle: @kerinokeefe
title: Nicosia 2013 Vulkà Bianco  (Etna)
variety: White Blend
winery: Nicosia' metadata={'source': './winemag-data-130k-v2.csv', 'row': 0}
1 page_content=': 1
country: Portugal
description: This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.
designation: Avidagos
points: 87
price: 15.0
province: Douro
region_1: 
region_2: 
taster_name: Roger Voss
taster_twitter_handle: @vossroger
title: Quinta dos Avidagos 2011 Avidagos Red (Douro)
varie

# 벡터스토어를 이용해 임베딩 & 적재
임베딩 모델 : "text-embedding-3-small"   
pinecone index_name : wine-reviews   
winemag-data-130k-v2.csv -> namespace : wine-review

In [ ]:
# 임베딩 모델 로딩
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model="text-embedding-3-small")

In [68]:
# 벡터스토어 적재
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore.from_documents(
    docs, 
    embedding, 
    index_name=("wine-reviews"), 
    namespace=("wine-review")
)

c:\Users\Admin\miniconda3\envs\prompt_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Admin\miniconda3\envs\prompt_env\Lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [74]:
# 질의하기
taste_query = "달콤한 맛을 느낄 수 있는 와인"
retrieved_docs = vector_store.similarity_search(taste_query, k=5, namespace="wine-review")
retrieved_docs

[Document(id='94a7d657-5f23-4574-884c-818e527b3857', metadata={'row': 47937.0, 'source': './winemag-data-130k-v2.csv'}, page_content=': 47937\ncountry: US\ndescription: As sweet as honey, with Lifesaver candy pineapple, golden apricot preserves, Meyer lemon custard pie and Asian spice flavors. Calls for very rich fare, such as scallops sautéed in butter, served with a creamy risotto.\ndesignation: \npoints: 88\nprice: 22.0\nprovince: California\nregion_1: Sonoma County\nregion_2: Sonoma\ntaster_name: \ntaster_twitter_handle: \ntitle: Wellington 2007 Roussanne (Sonoma County)\nvariety: Roussanne\nwinery: Wellington'),
 Document(id='501ded79-67f2-4164-a2e7-5b9b12482e38', metadata={'row': 45019.0, 'source': './winemag-data-130k-v2.csv'}, page_content=": 45019\ncountry: US\ndescription: Once again this designate amazes with a surfeit of beautifully integrated flavors. Mushrooms and baking spices adorn an explosion of strawberry pastries. There's plenty of ripe cherry fruit also, and a long

In [71]:
result = vector_store.similarity_search(
    response,
    k=5,
    namespace='wine-review'
)
result

[Document(id='390e2b08-6b36-43d9-bed3-74d10f162693', metadata={'row': 98767.0, 'source': './winemag-data-130k-v2.csv'}, page_content=": 98767\ncountry: Italy\ndescription: Here's a jammy, syrupy and dense expression of Primitivo with chewy layers of black cherry, leather, spice and tobacco. There's a sweet, overripe note in there as well that recalls strawberry comfiture and the wine is dense and chewy on the close.\ndesignation: Jèma\npoints: 87\nprice: \nprovince: Southern Italy\nregion_1: Salento\nregion_2: \ntaster_name: \ntaster_twitter_handle: \ntitle: L'Astore Masseria 2007 Jèma Primitivo (Salento)\nvariety: Primitivo\nwinery: L'Astore Masseria"),
 Document(id='166b7286-61b7-4d75-845d-3ece159de58d', metadata={'row': 24645.0, 'source': './winemag-data-130k-v2.csv'}, page_content=': 24645\ncountry: Italy\ndescription: This darkly colored Primitivo opens with sweet notes of Christmas spice, cinnamon, nutmeg, blackberry jam and plum cake. The wine is sticky and chewy on the close wi

# Wine Search

In [88]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

load_dotenv()

True

In [89]:
from dotenv import load_dotenv
import os

load_dotenv()

print(os.getenv("PINECONE_INDEX_NAME"))  # wine-reviews 출력되어야 함

wine-reviews


In [90]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

embedding = OpenAIEmbeddings(model='text-embedding-3-small')
vector_store = PineconeVectorStore(
    index_name=os.getenv("PINECONE_INDEX_NAME"),
    embedding=embedding,
    pinecone_api_key=os.getenv("PINECONE_API_KEY")
)

In [91]:
# vector store에서 현재 요리의 풍미와 맛과 유사한 와인 검색
taste_query =  "이 요리는 판차넬라 샐러드로, 신선한 토마토와 바질의 상큼함이 빵의 고소함과 어우러져 상쾌하고 풍부한 맛을 냅니다."
results = vector_store.similarity_search(
  taste_query, 
    k=5, 
    namespace=os.getenv("PINECONE_NAMESPACE")
)

results

[Document(id='120cc1f5-b58a-4ef4-9f9e-f1e6dd79db5f', metadata={'row': 12067.0, 'source': './winemag-data-130k-v2.csv'}, page_content=': 12067\ncountry: US\ndescription: Overtly sweet, this tastes like a dessert pastry turned into wine. The flavors are of orange, pineapple, vanilla-cream and buttered toast.\ndesignation: Back Seat Blonde\npoints: 82\nprice: 14.0\nprovince: California\nregion_1: Santa Ynez Valley\nregion_2: Central Coast\ntaster_name: \ntaster_twitter_handle: \ntitle: Coquelicot 2010 Back Seat Blonde White (Santa Ynez Valley)\nvariety: White Blend\nwinery: Coquelicot'),
 Document(id='d2300d18-c382-4bb0-92fe-45423de0bd77', metadata={'row': 115812.0, 'source': './winemag-data-130k-v2.csv'}, page_content=": 115812\ncountry: South Africa\ndescription: Vanilla toast and a compelling yeasty character on the nose entice. It's a little brighter on the palate, offering a balanced combination of minerality, acid and cream that lingers.\ndesignation: Nine Yards\npoints: 87\nprice: 

In [92]:
results = vector_store.similarity_search(
   response, 
    k=5, 
    namespace=os.getenv("PINECONE_NAMESPACE")
)

results

[Document(id='390e2b08-6b36-43d9-bed3-74d10f162693', metadata={'row': 98767.0, 'source': './winemag-data-130k-v2.csv'}, page_content=": 98767\ncountry: Italy\ndescription: Here's a jammy, syrupy and dense expression of Primitivo with chewy layers of black cherry, leather, spice and tobacco. There's a sweet, overripe note in there as well that recalls strawberry comfiture and the wine is dense and chewy on the close.\ndesignation: Jèma\npoints: 87\nprice: \nprovince: Southern Italy\nregion_1: Salento\nregion_2: \ntaster_name: \ntaster_twitter_handle: \ntitle: L'Astore Masseria 2007 Jèma Primitivo (Salento)\nvariety: Primitivo\nwinery: L'Astore Masseria"),
 Document(id='166b7286-61b7-4d75-845d-3ece159de58d', metadata={'row': 24645.0, 'source': './winemag-data-130k-v2.csv'}, page_content=': 24645\ncountry: Italy\ndescription: This darkly colored Primitivo opens with sweet notes of Christmas spice, cinnamon, nutmeg, blackberry jam and plum cake. The wine is sticky and chewy on the close wi

In [93]:
def search_wine(dish_flavor):
    results = vector_store.similarity_search(
        dish_flavor, 
        k=5, 
        namespace=os.getenv("PINECONE_NAMESPACE")
    )

    return {
        "dish_flavor": dish_flavor,
        "wine_reviews": "\n".join([doc.page_content for doc in results])
    }

In [94]:
runnable = RunnableLambda(search_wine)
response = runnable.invoke(taste_query)
print(response['dish_flavor'])
print(response['wine_reviews'])

이 요리는 판차넬라 샐러드로, 신선한 토마토와 바질의 상큼함이 빵의 고소함과 어우러져 상쾌하고 풍부한 맛을 냅니다.
: 12067
country: US
description: Overtly sweet, this tastes like a dessert pastry turned into wine. The flavors are of orange, pineapple, vanilla-cream and buttered toast.
designation: Back Seat Blonde
points: 82
price: 14.0
province: California
region_1: Santa Ynez Valley
region_2: Central Coast
taster_name: 
taster_twitter_handle: 
title: Coquelicot 2010 Back Seat Blonde White (Santa Ynez Valley)
variety: White Blend
winery: Coquelicot
: 115812
country: South Africa
description: Vanilla toast and a compelling yeasty character on the nose entice. It's a little brighter on the palate, offering a balanced combination of minerality, acid and cream that lingers.
designation: Nine Yards
points: 87
price: 38.0
province: Stellenbosch
region_1: 
region_2: 
taster_name: Susan Kostrzewa
taster_twitter_handle: @suskostrzewa
title: Jardin 2004 Nine Yards Chardonnay (Stellenbosch)
variety: Chardonnay
winery: Jardin
: 125634
country: 

# recommend_wine 

In [96]:
WINE_SOMMELIER_SYSTEM = """
        Persona:

        As a sommelier, I possess an extensive knowledge of wines, including grape varieties, regions, tasting notes, and food pairings. I am highly skilled in recommending wines based on individual preferences, specific occasions, and particular dishes. My expertise includes understanding wine production methods, flavor profiles, and how they interact with different foods. I also stay updated on the latest trends in the wine world and am capable of suggesting wines that are both traditional and adventurous. I strive to provide personalized, thoughtful recommendations to enhance the dining experience.

        Role:

        1. Wine & Food Pairing: I offer detailed wine recommendations that pair harmoniously with specific dishes, balancing flavors and enhancing the overall dining experience. Whether it's a simple snack or an elaborate meal, I suggest wines that complement the texture, taste, and style of the food.
        2. Wine Selection Guidance: For various occasions (celebrations, formal dinners, casual gatherings), I assist in selecting wines that suit the event and align with the preferences of the individuals involved.
        3. Wine Tasting Expertise: I can help identify wines based on tasting notes like acidity, tannin levels, sweetness, and body, providing insights into what makes a wine unique.
        4. Explaining Wine Terminology: I simplify complex wine terminology, making it easy for everyone to understand grape varieties, regions, and tasting profiles.
        5. Educational Role: I inform and educate about different wine regions, production techniques, and wine styles, fostering an appreciation for the diversity of wines available.

        Examples:

        - Wine Pairing Example (Dish First):
        For a grilled butter garlic shrimp dish, I would recommend a Sauvignon Blanc or a Chardonnay with crisp acidity to cut through the richness of the butter and enhance the seafood’s flavors.

        - Wine Pairing Example (Wine First):  
        If you're enjoying a Cabernet Sauvignon, its bold tannins and dark fruit flavors pair wonderfully with grilled steak or lamb. The richness of the meat complements the intensity of the wine.

        - Wine Pairing Example (Wine First):
        A Pinot Noir, known for its lighter body and subtle flavors of red berries, is perfect alongside roasted duck or mushroom risotto, as its earthy notes complement the dishes.

        - Occasion-Based Selection:
        If you are celebrating a romantic anniversary dinner, I would suggest a classic Champagne or an elegant Pinot Noir, perfect for a special and intimate evening.

        - Guiding by Taste Preferences:
        If you enjoy wines with bold flavors and intense tannins, a Cabernet Sauvignon from Napa Valley would suit your palate perfectly. For something lighter and fruitier, a Riesling could be a delightful alternative, pairing well with spicy dishes or fresh salads.
    """

In [97]:
def recommend_wine_chain(query):
    chat_template = ChatPromptTemplate.from_messages(
        [
        ('system', WINE_SOMMELIER_SYSTEM),
        ('human',"""
            와인 페이링 추천에 아래 요리/맛, 와인 리뷰만을 참고하여 한글로 답변해 주시기 바랍니다.
         
         요리/맛:
         {dish_flavor}

         와인 리뷰:
         {wine_reviews}
         
         """)
        ]
    )
    
    outputParser = StrOutputParser()
    
    chain = chat_template | llm | StrOutputParser()
    
    return chain

In [98]:
runnable_1 = RunnableLambda(describe_dish_flavor_chain)
runnable_2 = RunnableLambda(search_wine)
runnable_3 = RunnableLambda(recommend_wine_chain)

chain = runnable_1 | runnable_2 | runnable_3

In [105]:
# %pip install -qU grandalf

In [ ]:
# 음식 이미지 링크 > 음식 찾아서 음식맛 >  음식맛이 와인평에 있는 와인 검색 > 와인평 + 음식맛 > 소믈리에 메세지
chain.get_graph().print_ascii()

+----------------------------------+ 
| describe_dish_flavor_chain_input | 
+----------------------------------+ 
                  *                  
                  *                  
                  *                  
           +------------+            
           | ChatOpenAI |            
           +------------+            
                  *                  
                  *                  
                  *                  
          +-------------+            
          | search_wine |            
          +-------------+            
                  *                  
                  *                  
                  *                  
           +------------+            
           | ChatOpenAI |            
           +------------+            
                  *                  
                  *                  
                  *                  
  +-----------------------------+    
  | recommend_wine_chain_output |    
  +---------

In [103]:
response = chain.invoke(
    {
        "image_url" : "https://www.stockfood.com/Sites/StockFood/Documents/Homepage/News//en/16.jpg"
    }
)

In [104]:
response

'어떤 요리와 와인을 페어링할지 추천해드리겠습니다. 제공된 와인 리뷰를 참고하여 몇 가지 적합한 조합을 생각해보았습니다.\n\n1. **요리: 해산물 요리 (예: 홍합 스튜)**\n    - **와인 추천: Torii Mor 2011 Pinot Gris (Willamette Valley, US)**\n        - 이 와인은 얇고 신맛이 있으며, 레몬 주스의 맛이 특징입니다. 홍합 스튜와 같은 해산물 요리와 잘 어울리며, 해산물의 풍미를 강조할 수 있습니다. 가격은 약 18,000원으로 합리적입니다.\n\n2. **요리: 아시아 요리 (예: 향신료가 들어간 중국식 요리)**\n    - **와인 추천: Rieflé 2005 Pinot Gris (Alsace, France)**\n        - 이 와인은 medium-sweet한 맛이 특징이며, 리치와 향신료의 풍미가 강조되어 있습니다. 아시아 요리의 복잡한 맛과 잘 매칭될 수 있습니다.\n\n3. **요리: 치킨 요리 (예: 허브로 구운 치킨)**\n    - **와인 추천: Yealands 2010 Pinot Noir (Marlborough, New Zealand)**\n        - 이 피노 누아는 체리와 장미향이 매력적이며, 부드러운 질감을 지니고 있습니다. 허브로 구운 치킨과 잘 어울리며, 지금 마셔도 좋습니다. 가격은 약 17,000원입니다.\n\n위의 추천 사항들이 도움이 되셨으면 좋겠습니다. 궁금한 점이나 추가하고 싶은 사항이 있으시면 언제든지 말씀해 주세요!'